In [1]:
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime, timedelta

In [2]:
days = 100 # to adjust for backup data loss or else 90
past_date = datetime.now() - timedelta(days=days)
pdate = past_date.strftime('%Y-%m-%d')
pdatetime = datetime.strptime(pdate, '%Y-%m-%d').strftime('%Y-%m-%d %H:%M:%S')
business = 100005

In [3]:
db_config = {'user': 'XXXXXX','password': 'XXXXXX','host': 'XXXXXX','port': '5432', 'database': 'XXXXXX'}
engine = create_engine(f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

In [4]:
query_caitem = f"""
        SELECT caitem.xitem, caitem.xdesc
        FROM caitem
        WHERE caitem.zid = %(business)s
    """
params_caitem = {'business': business}

query_imtrn = f"""
        SELECT imtrn.xitem,imtrn.xyear, imtrn.xper, imtrn.xdate,imtrn.xqty, imtrn.xval, imtrn.xdocnum
        FROM imtrn 
        WHERE imtrn.zid = %(business)s
        AND imtrn.xdate >= %(date)s
        AND imtrn.xdocnum LIKE %(docnum)s
    """
params_imtrn = {'business': business, 'date': pdate, 'docnum':'DO--%'}
caitem_data = pd.read_sql(query_caitem, con=engine, params=params_caitem)
imtrn_data = pd.read_sql(query_imtrn, con=engine, params=params_imtrn)

In [5]:
query_opddt = f"""
        SELECT opddt.ztime, opddt.xitem, opddt.xordernum, opddt.xdornum
        FROM opddt
        WHERE opddt.zid = %(business)s
        AND opddt.ztime >= %(datetime)s
    """
params_opddt = {'business': business, 'datetime': pdatetime}

query_opodt = f"""
        SELECT opodt.xitem, opodt.xlineamt, opodt.xordernum, opord.xdate
        FROM opodt
        JOIN opord
        ON opodt.xordernum = opord.xordernum
        WHERE opodt.zid = %(business)s
        AND opord.zid = %(business)s
        AND opord.xdate >= %(date)s
    """
params_opodt = {'business': business, 'date': pdate}

opddt_data = pd.read_sql(query_opddt, con=engine, params=params_opddt)
opodt_data = pd.read_sql(query_opodt, con=engine, params=params_opodt)

In [6]:
imtrn_data = imtrn_data.groupby(['xitem', 'xdocnum']).agg({
    'xqty': 'sum',      
    'xval': 'sum',       
    'xyear': 'first',    
    'xdate': 'first',    
    'xper': 'first'      
}).reset_index()

opodt_data_aggregated = opodt_data.groupby(['xitem', 'xordernum']).agg({
    'xlineamt': 'sum',
    'xdate': 'first'
}).reset_index()

In [7]:
opddt_data = opddt_data.drop(columns=['ztime'])
opddt_data = opddt_data.drop_duplicates(subset=['xitem', 'xordernum', 'xdornum'])

In [8]:
# merge all the data and aggregate duplicates (discounts and bundle offers)
merged_1 = pd.merge(imtrn_data, opddt_data, left_on=['xitem','xdocnum'], right_on=['xitem','xdornum'], how='left')
merged_2 = pd.merge(merged_1, opodt_data, left_on=['xitem','xordernum'], right_on=['xitem','xordernum'], how='left')

In [9]:
merged_2 = merged_2.groupby(['xitem', 'xdocnum','xordernum','xdornum']).agg({
    'xqty': 'first',      
    'xval': 'first',       
    'xyear': 'first',
    'xdate_x': 'first',
    'xper': 'first',
    'xlineamt': 'sum',    
    'xdate_y': 'first'      
}).reset_index()

In [10]:
final_merged = pd.merge(merged_2, caitem_data, left_on='xitem', right_on='xitem', how='left')
sales_data = final_merged[['xitem', 'xdesc', 'xdocnum','xyear','xper', 'xqty', 'xval', 'xlineamt']]

In [11]:
sales_data = sales_data[sales_data['xlineamt'] > 0]

In [12]:
# monthly sales per product
sales_data_m = sales_data.groupby(['xitem', 'xdesc','xyear','xper',]).agg({
    'xqty': 'sum',      
    'xval': 'sum',       
    'xlineamt': 'sum'   
}).reset_index()

In [13]:
# Average Sales per month for each product
sales_data['Profit'] = sales_data['xlineamt']

In [14]:
query_moord = f"""
        SELECT moord.xdatemo, moord.xmoord, moord.xitem as goods, caitem.xdesc, moord.xqtyprd
        FROM moord
        JOIN caitem
        ON moord.xitem = caitem.xitem
        WHERE moord.zid = %(business)s
        AND caitem.zid = %(business)s
        AND moord.xdatemo >= %(date)s
    """
params_moord = {'business': business, 'date': pdate}
moord_data = pd.read_sql(query_moord, con=engine, params=params_moord)

In [15]:
query_moodt = f"""
        SELECT  moodt.ztime, moodt.xmoord, moodt.xitem as raw, caitem.xdesc, moodt.xqtyord, moodt.xrate
        FROM moodt 
        JOIN caitem
        ON moodt.xitem = caitem.xitem
        WHERE moodt.zid = %(business)s
        AND caitem.zid = %(business)s
        AND moodt.ztime >= %(datetime)s
    """
params_moodt = {'business': business, 'datetime': pdatetime}
moodt_data = pd.read_sql(query_moodt, con=engine, params=params_moodt)

In [16]:
moodt_data

,ztime,xmoord,raw,xdesc,xqtyord,xrate
0,2024-07-02 16:56:49.884,MO--003465,RZ000273,SDCL0001,0.780,59.67
1,2024-07-02 16:56:49.884,MO--003465,RZ000002,IPPL4250,0.065,347.83
2,2024-07-02 16:56:49.931,MO--003465,RZ000409,Shrink Poly 5 Liter,13.000,2.42
3,2024-07-02 16:56:49.868,MO--003465,RZ000282,ASSA0001,1.950,210.16
4,2024-07-02 16:56:49.884,MO--003465,RZ000309,SDBZ0001,0.065,271.25
...,...,...,...,...,...,...
4217,2024-09-26 11:01:37.755,MO--003796,RZ000454,NP000010,0.055,450.01
4218,2024-09-26 11:01:37.737,MO--003796,RZ000273,SDCL0001,0.866,60.79
4219,2024-09-26 11:01:37.744,MO--003796,RZ000235,SLES0001,4.125,172.00
4220,2024-09-26 11:01:37.750,MO--003796,RZ000160,PRFLVD01,0.126,3993.62


In [17]:
moord_data

,xdatemo,xmoord,goods,xdesc,xqtyprd
0,2024-07-02,MO--003470,FZ000080,Liquid Hand soap 5 Litre (Super),30.0
1,2024-07-02,MO--003465,FZ000190,Floor Cleaner (Lemon) 5 Litre (Super),13.0
2,2024-07-02,MO--003463,FZ000185,White Phenyle 3 Litre,40.0
3,2024-07-02,MO--003467,FZ000041,Liquid Hand soap 5 Litre (Supreme),14.0
4,2024-07-02,MO--003466,FZ000191,Floor Cleaner (Lavender) 5 Litre (Super),12.0
...,...,...,...,...,...
327,2024-09-26,MO--003793,FZ000005,Kitchen Degreaser New,300.0
328,2024-09-26,MO--003792,FZ000028,Furniture Cleaner,360.0
329,2024-09-26,MO--003795,FZ000037,Dish washing liquid 5 Litre,70.0
330,2024-09-26,MO--003794,FZ000081,Liquid Hand soap 5 Litre (Basic),70.0


In [18]:
merged_mo = pd.merge(moodt_data, moord_data, left_on='xmoord', right_on='xmoord', how='left')

In [19]:
merged_mo.to_excel('total_mo.xlsx')

In [20]:
merged_mo_agg = merged_mo.groupby(['raw','goods']).agg({
    'xqtyord': 'sum',      
    'xqtyprd': 'sum',       
    'xrate': 'mean'     
}).reset_index()

In [21]:
merged_mo_agg = merged_mo_agg.sort_values(by='goods', ascending=True)

In [22]:
merged_mo_agg['xqty/unit'] = merged_mo_agg['xqtyord']/merged_mo_agg['xqtyprd']

In [23]:
merged_mo_unit = merged_mo_agg[['goods','raw','xqty/unit', 'xrate']]

In [24]:
merged_mo_unit.to_excel('finished_raw_ratio_rate.xlsx')

In [25]:
sales_data_agg = sales_data_m.groupby(['xitem','xdesc']).agg({
    'xqty': lambda x: round(x.mean(), 2),     
    'xval': lambda x: round(x.mean(), 2),       
    'xlineamt': lambda x: round(x.mean(), 2)  
}).reset_index()

In [26]:
sales_data_agg.to_excel('Average_sales_product.xlsx')

In [27]:
sales_data_agg.columns

Index(['xitem', 'xdesc', 'xqty', 'xval', 'xlineamt'], dtype='object')

In [28]:
merged_mo_unit.columns

Index(['goods', 'raw', 'xqty/unit', 'xrate'], dtype='object')

In [29]:
products = sales_data_agg['xitem'].unique()
average_sales = dict(zip(sales_data_agg['xitem'], sales_data_agg['xqty']))

In [31]:
raw_material_rate = merged_mo_unit.groupby(['raw']).agg({
    'xrate': lambda x: round(x.mean(), 2),      
}).reset_index()

In [35]:
raw_material_rate.to_excel('raw_rate.xlsx')